<a href="https://colab.research.google.com/github/sarahbraia/dissertation_methodology_analysis_ITA/blob/main/Artigo_E_tech_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#APPLICATION OF MACHINE LEARNING FOR STANDARD TIME ESTIMATION IN INDUSTRIAL CABLE ASSEMBLY

## 1. Installing and importing libraries

In [24]:
#Importando os dados do google drive
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Loading data

In [25]:
# Reading files from Google Drive
base_path = '/content/drive/MyDrive/ITA -DISSERTAÇÃO/cabling_time_study_data.xlsx'
df = pd.read_excel(base_path)

# Preview of each dataset
df.head()

,OP,ETAPA,ID_OPERACAO,QTD,SETUP_HORA(S.H),EXECUCAO_HORA(E.H),TEMPO_TOTAL(T.T)
0,41810753,decapar os condutores,Decapar,1,00:02:34,00:02:28,00:05:02
1,41810753,corte do ajuste do tamanho das extremidades do...,Cortar,1,00:02:48,00:00:40,00:03:28
2,41810753,crimpagem,Crimpar,1,00:01:11,00:00:00,00:01:11
3,41810753,ABERTURA DE ANDON - CONECTOR PAGO ERRADO,Abrir chamado,1,00:09:00,00:04:29,00:13:29
4,41810753,Analise do Andon pelo hardwer,Abrir chamado,1,00:07:00,00:00:00,00:07:00


### Application of Micromovement Study in Cable Assembly

In [26]:
# Convert time columns from string to datetime format
df['SETUP_HORA(S.H)'] = pd.to_datetime(df['SETUP_HORA(S.H)'], format='%H:%M:%S', errors='coerce')
df['EXECUCAO_HORA(E.H)'] = pd.to_datetime(df['EXECUCAO_HORA(E.H)'], format='%H:%M:%S', errors='coerce')
df['TEMPO_TOTAL(T.T)'] = pd.to_datetime(df['TEMPO_TOTAL(T.T)'], format='%H:%M:%S', errors='coerce')

# Replace missing values with 00:00:00
df['SETUP_HORA(S.H)'] = df['SETUP_HORA(S.H)'].fillna(pd.to_datetime('00:00:00'))
df['EXECUCAO_HORA(E.H)'] = df['EXECUCAO_HORA(E.H)'].fillna(pd.to_datetime('00:00:00'))
df['TEMPO_TOTAL(T.T)'] = df['TEMPO_TOTAL(T.T)'].fillna(pd.to_datetime('00:00:00'))

# Function to convert time to seconds
def time_to_seconds(t):
    return t.hour * 3600 + t.minute * 60 + t.second

# Apply the function to extract seconds from each time column
df['SETUP_SECONDS'] = df['SETUP_HORA(S.H)'].dt.time.apply(time_to_seconds)
df['EXECUTION_SECONDS'] = df['EXECUCAO_HORA(E.H)'].dt.time.apply(time_to_seconds)
df['TOTAL_SECONDS'] = df['TEMPO_TOTAL(T.T)'].dt.time.apply(time_to_seconds)

# Preview the result
df[['ID_OPERACAO', 'SETUP_SECONDS', 'EXECUTION_SECONDS', 'TOTAL_SECONDS']].head(15)

,ID_OPERACAO,SETUP_SECONDS,EXECUTION_SECONDS,TOTAL_SECONDS
0,Decapar,154,148,302
1,Cortar,168,40,208
2,Crimpar,71,0,71
3,Abrir chamado,540,269,809
4,Abrir chamado,420,0,420
5,Preparar bancada,116,41,157
6,Organizar os fios,120,25,145
7,Crimpar,0,35,35
8,Medir tamanho do cabo,0,19,19
9,Cortar,117,15,132


In [27]:
# Dictionary for ID and English operation name mapping
operation_mapping = {
    'Abrir chamado': (1, 'Open service request'),
    'Preparar bancada': (2, 'Prepare workstation'),
    'Medir tamanho do cabo': (3, 'Measure cable length'),
    'Marcar': (3, 'Measure cable length'),
    'Cortar': (4, 'Cut'),
    'Decapar': (5, 'Strip wire'),
    'Crimpar': (6, 'Crimp'),
    'Estanhar': (7, 'Tin wires'),
    'Soldar': (8, 'Solder'),
    'Limpar excesso de solda': (9, 'Remove excess solder'),
    'Soprador térmico': (10, 'Apply heat gun'),
    'Fabricar adesivo': (11, 'Produce adhesive'),
    'Inserir': (12, 'Insert'),
    'Organizar os fios': (13, 'Organize wires'),
    'Montar Conector': (14, 'Assemble connector'),
    'Teste de continuidade': (15, 'Continuity test')
}

# Create the new columns using the mapping
df['ID'] = df['ID_OPERACAO'].map(lambda x: operation_mapping.get(x, (None, None))[0])
df['Operation_ID_(EN)'] = df['ID_OPERACAO'].map(lambda x: operation_mapping.get(x, (None, None))[1])

# Rename the column
df.rename(columns={'ID_OPERACAO': 'ID_OPERAÇÃO_(PT)'}, inplace=True)

# order by operation ID
df_sorted = df.sort_values(by='ID')

# Visualizar resultado
df_sorted[['ID', 'ID_OPERAÇÃO_(PT)', 'Operation_ID_(EN)', 'TOTAL_SECONDS']].head(15)


,ID,ID_OPERAÇÃO_(PT),Operation_ID_(EN),TOTAL_SECONDS
3,1,Abrir chamado,Open service request,809
4,1,Abrir chamado,Open service request,420
5,2,Preparar bancada,Prepare workstation,157
642,2,Preparar bancada,Prepare workstation,255
392,2,Preparar bancada,Prepare workstation,203
356,2,Preparar bancada,Prepare workstation,214
54,2,Preparar bancada,Prepare workstation,214
379,2,Preparar bancada,Prepare workstation,605
288,2,Preparar bancada,Prepare workstation,498
543,2,Preparar bancada,Prepare workstation,300


#### Refined cabling operation

In [28]:
# Group by ID and Operation_ID_(EN) and count occurrences
distinct_counts = (
    df.groupby(['ID', 'Operation_ID_(EN)'])
    .size()
    .reset_index(name='COUNT')
    .sort_values(by='ID')
)

# Display the result
distinct_counts


,ID,Operation_ID_(EN),COUNT
0,1,Open service request,2
1,2,Prepare workstation,15
2,3,Measure cable length,84
3,4,Cut,55
4,5,Strip wire,109
5,6,Crimp,30
6,7,Tin wires,47
7,8,Solder,70
8,9,Remove excess solder,26
9,10,Apply heat gun,56
